# This is our BASE model. Please do not edit this original copy.

## Dataset EDA

In [ ]:
!pip install opendatasets
!pip install tensorflow
!pip install opencv-python

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import glob
import random
import os
import re
from os import listdir
from sklearn.metrics import classification_report
import tensorflow as tf
import keras.utils as image
from tensorflow.keras.utils import to_categorical

In [ ]:
import opendatasets as od

#when prompted, you need to fill in your Kaggle username and key.
#to obtain these, sign into Kaggle, click your profile picture > Settings > Create New API Token
# this will downlolad a file called kaggle.json that will include your username and key

od.download("https://www.kaggle.com/datasets/paultimothymooney/breast-histopathology-images")

In [ ]:
#check to see if the dataset has loaded on your runtime
dataset_path = './breast-histopathology-images/IDC_regular_ps50_idx5/'

img_paths = glob.glob(os.path.join(dataset_path, '**', '*.png'), recursive=True)

for img in img_paths[:3]:
    print(img)

# Quick EDA

In [ ]:
neg_imgs = [] #Class = 0 : non-cancerous
pos_imgs = [] #Class = 1 : cancerous

for img in img_paths:
    if img[-5] == '0' :
        neg_imgs.append(img)

    elif img[-5] == '1' :
        pos_imgs.append(img)

print(f"IDC (-): {len(neg_imgs)}")
print(f"IDC (+): {len(pos_imgs)}")
#should print:
# IDC (-): 198738
# IDC (+): 78786

In [ ]:
plt.figure(figsize = (15, 15))
some_non = np.random.randint(0, len(neg_imgs), 18)
some_can = np.random.randint(0, len(pos_imgs), 18)

s = 0
for num in some_non:
    img_obj = image.load_img((neg_imgs[num]), target_size=(100, 100))
    img_array = image.img_to_array(img_obj)

    plt.subplot(6, 6, 2*s+1)
    plt.axis('off')
    plt.title('IDC (-)')
    plt.imshow(img_array.astype('uint8'))
    s += 1

s = 1
for num in some_can:
    img_obj = image.load_img((pos_imgs[num]), target_size=(100, 100))
    img_array = image.img_to_array(img_obj)

    plt.subplot(6, 6, 2*s)
    plt.axis('off')
    plt.title('IDC (+)')
    plt.imshow(img_array.astype('uint8'))
    s += 1

# Patient-level Split

In [ ]:
def parse_filename(path):
    '''
    Function: parse patient IDs and labels from filenames
    Path: image path
    '''
    filename = os.path.basename(path)
    match = re.match(r"(\d+_idx\d+)_x\d+_y\d+_class(\d)\.png", filename)
    if match:
        patient_id = match.group(1)
        label = int(match.group(2))
        return patient_id, label
    else:
        return None, None

data = []
for path in img_paths:
    pid, label = parse_filename(path)
    if pid is not None:
        data.append([path, pid, label])

df = pd.DataFrame(data, columns=["path", "patient_id", "label"])

In [ ]:
num_patients = df['patient_id'].nunique()
print("Number of unique patients:", num_patients)


In [ ]:
from sklearn.model_selection import train_test_split

# 0 = patient has only IDC-, 1 = patient has at least one IDC+
patient_labels = df.groupby('patient_id')['label'].max()

RSEED = 5526  # reproducibility

# 60% train, 40% temp (val+test), stratified by patient
train_patients, temp_patients = train_test_split(
    patient_labels.index,
    test_size=0.40,
    stratify=patient_labels,
    random_state=RSEED
)

# Split remaining 40% into 20% val and 20% test, again stratified
temp_labels = patient_labels.loc[temp_patients]
val_patients, test_patients = train_test_split(
    temp_patients,
    test_size=0.50,
    stratify=temp_labels,
    random_state=RSEED
)

# Build dataframes
train_df = df[df['patient_id'].isin(train_patients)]
val_df   = df[df['patient_id'].isin(val_patients)]
test_df  = df[df['patient_id'].isin(test_patients)]

# shuffle each dataframe once to mix label order before TensorFlow loads them
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
val_df   = val_df.sample(frac=1, random_state=42).reset_index(drop=True)
test_df  = test_df.sample(frac=1, random_state=42).reset_index(drop=True)


# Sanity checks
print(f"✅ Train: {len(train_df)} images, {len(train_patients)} patients")
print(f"✅ Val:   {len(val_df)} images, {len(val_patients)} patients")
print(f"✅ Test:  {len(test_df)} images, {len(test_patients)} patients")

# Confirm no overlap
assert set(train_patients).isdisjoint(val_patients)
assert set(train_patients).isdisjoint(test_patients)
assert set(val_patients).isdisjoint(test_patients)
print("✅ No patient overlap between splits.")

# Confirm both classes exist in every split
for name, subset in (("Train", train_df), ("Val", val_df), ("Test", test_df)):
    counts = subset['label'].value_counts().to_dict()
    print(f"{name} class counts:", counts)
    assert 0 in counts and 1 in counts, f"{name} is missing a class!"


# Loading Images

Note:

When you run something in Colab, you’re not running it on your laptop... you’re running it on a big physical computer (a server) inside Google’s data center.

Your Colab notebook doesn’t own that whole machine. instead, Google creates a container (like a tiny virtual sandbox) inside the host.

In [ ]:
from psutil import virtual_memory

#total physical RAM of the Google Cloud host machine that your Colab container is running on
ram = virtual_memory()
print(f"Used: {ram.used / 1e9:.2f} GB")
print(f"Available: {ram.available / 1e9:.2f} GB")
print(f"Total: {ram.total / 1e9:.2f} GB")


In [ ]:
import psutil

#actual available RAM your colab container is using.

def get_container_memory_limit():
    paths = [
        "/sys/fs/cgroup/memory.max",  # cgroup v2
        "/sys/fs/cgroup/memory/memory.limit_in_bytes"  # cgroup v1 (older Colab)
    ]

    for p in paths:
        if os.path.exists(p):
            val = open(p).read().strip()
            if val == "max":
                # Means no limit at host level, fall back to actual available memory
                mem = psutil.virtual_memory().total
                return mem / (1024 ** 3)
            else:
                return int(val) / (1024 ** 3)
    return None

limit_gb = get_container_memory_limit()
if limit_gb:
    print(f"Actual usable container RAM: {limit_gb:.2f} GB")
else:
    print("Could not detect memory limit.")


In [ ]:
#make sure you are running on T4 GPU

gpu_avail = tf.config.list_physical_devices('GPU')
print("Is the GPU available?", len(gpu_avail) > 0)

In [ ]:
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

AUTOTUNE = tf.data.AUTOTUNE #automatically figure out the optimal number of threads or buffer for loading, processing, getting the data
IMG_SIZE = (224, 224)
BATCH_SIZE = 32  #160284 training images / 32 batches = 5009 images/batch

#A note on the WARNING you may receive ("Transparent hugepages are not enabled")
#Transparent hugepages are a Linux kernal feature that helps CPUs and TPUs handle large memory pages more efficiently
#It has no effect on Tensorflow training cause this happens on the GPU

In [ ]:
def load_and_preprocess(path, label):
    img = tf.io.read_file(path)                 # Reads image bytes from disk (lazy. not all at once)
    img = tf.image.decode_png(img, channels=3)  # Decodes bytes -> H×W×3 tensor (RGB). (Use decode_image for mixed formats.)
    img = tf.image.resize(img, IMG_SIZE)        # Resizes to 224×224 so it matches the model’s expected input
    img = preprocess_input(img)                 # Applies EfficientNetV2 normalization (not just /255)
    label = tf.one_hot(label, depth=2)          # Converts class index (0/1) -> one-hot vector [1,0] / [0,1]
    return img, label


In [ ]:
def make_dataset(df, shuffle=True):
    paths = df['path'].values
    labels = df['label'].values

    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(load_and_preprocess, num_parallel_calls=AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(buffer_size=1000)

    ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return ds

train_ds = make_dataset(train_df, shuffle=True)
val_ds = make_dataset(val_df, shuffle=False)
test_ds = make_dataset(test_df, shuffle=False)


In [ ]:
def count_classes(dataset):
    counts = np.array([0, 0])
    for _, y in dataset:
        y = tf.argmax(y, axis=1).numpy()
        counts += np.bincount(y, minlength=2)
    return counts

#ensuring our training, validation, and test sets are being cached correctly
for split, ds in zip(["train", "val", "test"], [train_ds, val_ds, test_ds]):
    print(split, count_classes(ds))

## Creating the Model

In [ ]:
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D,MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam, SGD
from keras.metrics import binary_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix
import itertools
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras import regularizers, metrics


In [ ]:
# Define image shape and class count
IMG_SIZE = (224, 224)
CHANNELS = 3
IMG_SHAPE = (IMG_SIZE[0], IMG_SIZE[1], CHANNELS)
CLASS_COUNT = 2

# Load EfficientNetV2S pretrained on ImageNet
base_model = tf.keras.applications.EfficientNetV2S(
    include_top=False,
    weights="imagenet",
    input_shape=IMG_SHAPE,
    pooling='avg'   # 'avg' is more stable for fine-tuning
)

#Unfreeze top layers for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-50]:  # freeze all but last ~50 layers
    layer.trainable = False

# Classification head
model = tf.keras.Sequential([
    base_model,
    # tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001),
    tf.keras.layers.Dense(
        256,
        kernel_regularizer=regularizers.l2(1e-4),
        # activity_regularizer=regularizers.l1(0.006),
        # bias_regularizer=regularizers.l1(0.01),
        activation='relu'
    ),
    tf.keras.layers.Dropout(rate=0.4, seed=888),
    tf.keras.layers.Dense(CLASS_COUNT, activation='softmax')
])

model.summary()


## Training the Model

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=1e-4),
    loss="categorical_crossentropy"
)

In [ ]:
es = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

In [ ]:
# Check label shapes
for x_batch, y_batch in val_ds.take(1):
    print("Batch X shape:", x_batch.shape)
    print("Batch Y shape:", y_batch.shape)
    # print("X unique vectors:", np.unique(x_batch.numpy(), axis=0))
    print("Y unique vectors:", np.unique(y_batch.numpy(), axis=0))


In [ ]:
for batch_idx, (x, y) in enumerate(val_ds.take(3)):
    print(f"Batch {batch_idx}: {np.sum(np.argmax(y.numpy(), axis=1)==1)} positives out of {y.shape[0]}")


In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=60,
    callbacks=[es],
    verbose=1
)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import (
    BaggingClassifier,
    RandomForestClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier,
    StackingClassifier,
    VotingClassifier
)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix
from tensorflow.keras.models import Model
import tensorflow as tf # Ensure tensorflow is imported in this context

# ---------- Feature Extraction from CNN Layer ----------
def extract_features(model, dataset, layer_name="dense"):
    # Ensure the model is built by performing a dummy call if it hasn't been already.
    # This should technically be handled by model.fit() but can be an issue when reusing a Sequential model's graph.
    if not model.built:
        # Find a batch from the dataset to build the model
        for x_batch, _ in dataset.take(1):
            model(x_batch)
            break

    # Use the input of the main model for the feature extractor.
    # This is more robust for Sequential models than directly using model.layers[0].input.
    input_tensor_for_extractor = model.input
    output_tensor_for_extractor = model.get_layer(layer_name).output

    intermediate_layer_model = Model(inputs=input_tensor_for_extractor,
                                     outputs=output_tensor_for_extractor)
    features, labels = [], []
    for batch_x, batch_y in dataset:
        feats = intermediate_layer_model.predict(batch_x, verbose=0)
        features.append(feats)
        labels.append(batch_y.numpy())
    X = np.concatenate(features)
    Y = np.concatenate(labels)
    return X, np.argmax(Y, axis=1)

# ---------- Ensemble Training and Evaluation ----------
def train_and_evaluate_ensembles(X_train, y_train, X_test, y_test):
    results = {}

    base_svc = SVC(probability=True)
    base_lr = LogisticRegression(max_iter=5000)

    models = {
        "Bagging SVM": BaggingClassifier(estimator=SVC(), n_estimators=10), # 'base_estimator' is deprecated, use 'estimator'
        "Random Forest": RandomForestClassifier(n_estimators=100),
        "AdaBoost": AdaBoostClassifier(n_estimators=100),
        "Gradient Boosting": GradientBoostingClassifier(n_estimators=100),
        "Stacking": StackingClassifier(
            estimators=[('svc', base_svc), ('lr', base_lr)],
            final_estimator=LogisticRegression()
        ),
        "Voting": VotingClassifier(
            estimators=[('svc', base_svc), ('lr', base_lr), ('rf', RandomForestClassifier(n_estimators=50))],
            voting='soft'
        )
    }

    for name, clf in models.items():
        clf.fit(X_train, y_train)
        preds = clf.predict(X_test)
        acc = accuracy_score(y_test, preds)
        prec = precision_score(y_test, preds)
        rec = recall_score(y_test, preds)
        results[name] = {"accuracy": acc, "precision": prec, "recall": rec}

    return results, models

# ---------- Plotting ----------
def plot_metrics(results):
    metrics = ["accuracy", "precision", "recall"]
    for metric in metrics:
        values = [v[metric] for v in results.values()]
        labels = list(results.keys())
        plt.figure(figsize=(10, 5))
        sns.barplot(x=labels, y=values)
        plt.title(f"Model Comparison - {metric.title()}")
        plt.ylabel(metric.title())
        plt.xticks(rotation=30)
        plt.ylim(0, 1.05)
        plt.tight_layout()
        plt.show()

# ---------- Confusion Matrix Evaluation ----------
def plot_conf_matrix(y_true, y_pred, title):
    conf_mtx = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(7, 5))
    sns.heatmap(conf_mtx, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['Predicted IDC-', 'Predicted IDC+'],
                yticklabels=['Actual IDC-', 'Actual IDC+'])
    plt.title(f'Confusion Matrix - {title}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

    conf_mtx_percent = conf_mtx.astype(float) / conf_mtx.sum(axis=1)[:, np.newaxis] * 100
    plt.figure(figsize=(7, 5))
    sns.heatmap(conf_mtx_percent, annot=True, fmt=".2f", cmap='Blues', cbar=False,
                xticklabels=['Predicted IDC-', 'Predicted IDC+'],
                yticklabels=['Actual IDC-', 'Actual IDC+'])
    plt.title(f'Confusion Matrix (%) - {title}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

In [ ]:
!pip install tensorflow
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import (
    BaggingClassifier,
    RandomForestClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier,
    StackingClassifier,
    VotingClassifier
)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Model

# Before calling extract_features, ensure the model is built by making a dummy prediction
# This ensures that model.input is defined for the functional API.
try:
    for x_batch, _ in train_ds.take(1):
        _ = model.predict(x_batch)
        break # Only need one batch to build it
except NameError:
    print("\nError: 'train_ds' is not defined. Please ensure that the cells defining 'df', 'train_df', 'val_df', 'test_df', 'make_dataset', 'load_and_preprocess', 'IMG_SIZE', 'BATCH_SIZE', 'AUTOTUNE', and 'preprocess_input' have been executed.")
    raise # Re-raise the error to stop execution

# Get the actual name of the 256-unit Dense layer (which is at index 1 in the Sequential model)
feature_layer_name = model.layers[1].name

X_train, y_train = extract_features(model, train_ds, layer_name=feature_layer_name)
X_test, y_test = extract_features(model, test_ds, layer_name=feature_layer_name)

results, models = train_and_evaluate_ensembles(X_train, y_train, X_test, y_test)
plot_metrics(results)

for name, clf in models.items():
    y_pred = clf.predict(X_test)
    print(f"\n{name} — Classification Report:")
    print(classification_report(y_test, y_pred, target_names=['IDC-', 'IDC+']))
    print("Accuracy:", accuracy_score(y_test, y_pred))
    plot_conf_matrix(y_test, y_pred, title=name)

In [ ]:
preds = model.predict(val_ds.take(10))
print("Preds shape:", preds.shape)
print("Unique rows:", np.unique(np.round(preds, 2), axis=0)[:5])
print("Pred range:", preds.min(), preds.max())


In [ ]:
for layer in model.layers:
    print(layer.name, layer.trainable)


## Evaluating Model Performance

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Generate predictions on test set ---
y_true = []
y_pred = []

for x_batch, y_batch in test_ds:
    preds = model.predict(x_batch, verbose=0)          # (batch,2)
    pred_labels = np.argmax(preds, axis=1)             # convert to 0/1
    true_labels = np.argmax(y_batch.numpy(), axis=1)   # convert to 0/1

    y_pred.extend(pred_labels)
    y_true.extend(true_labels)

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# --- Confusion Matrix ---
conf_mtx = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(7, 5))
sns.heatmap(conf_mtx, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Predicted IDC-', 'Predicted IDC+'],
            yticklabels=['Actual IDC-', 'Actual IDC+'])
plt.title('Confusion Matrix (Test)')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# --- Percent Confusion Matrix ---
conf_mtx_percent = conf_mtx.astype(float) / conf_mtx.sum(axis=1)[:, np.newaxis] * 100

plt.figure(figsize=(7, 5))
sns.heatmap(conf_mtx_percent, annot=True, fmt=".2f", cmap='Blues', cbar=False,
            xticklabels=['Predicted IDC-', 'Predicted IDC+'],
            yticklabels=['Actual IDC-', 'Actual IDC+'])
plt.title('Confusion Matrix (%)')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# --- Manual Metric Report ---
print("\nClassification Report:")
print(classification_report(y_true, y_pred, digits=4))

# --- Manual Accuracy ---
print("Accuracy:", accuracy_score(y_true, y_pred))


In [ ]:
def show_test_image(dataset, index=0, model=None):
    """Display one sample from a tf.data.Dataset and print true/pred labels."""
    # fetch the exact index-th element from the unbatched dataset
    img = label = None
    for i, (x, y) in enumerate(dataset.unbatch()):
        if i == index:
            img = x.numpy()          # preprocessed image (float)
            label = int(np.argmax(y.numpy()))
            break
    if img is None:
        print(f"Index {index} out of range.")
        return

    # ---- prediction (use the preprocessed img) ----
    pred = None
    if model is not None:
        probs = model.predict(img[None, ...], verbose=0)
        pred = int(np.argmax(probs, axis=1)[0])

    # ---- fix scaling for display ----
    disp = img.copy()
    if disp.dtype != np.uint8:
        if disp.max() > 1.5:         # floats in [0..255] -> [0..1]
            disp = disp / 255.0
        elif disp.min() < 0:         # e.g., [-1..1] -> [0..1]
            m, M = disp.min(), disp.max()
            disp = (disp - m) / (M - m + 1e-8)

    plt.figure(figsize=(3,3))
    plt.imshow(disp)
    title_true = "IDC+" if label == 1 else "IDC-"
    title_pred = "IDC+" if (pred == 1) else "IDC-" if (pred is not None) else "—"
    plt.title(f"True: {title_true} | Pred: {title_pred}")
    plt.axis("off")
    plt.show()

# Example
show_test_image(test_ds, index=200, model=model)
show_test_image(test_ds, index=4563, model=model)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
save_dir = "/content/drive/MyDrive/CSCI5526/SavedModels"
model.save(f"{save_dir}/base_model.h5")


In [ ]:
!ls -lh "/content/drive/MyDrive/CSCI5526"
!ls -lh "/content/drive/MyDrive/CSCI5526/SavedModels"
